In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.feature_extraction.text import TfidfTransformer
from keras.regularizers import l2, activity_l2
from sklearn.neighbors import KNeighborsClassifier
from keras.callbacks import Callback
import xgboost as xgb

class NBatchLogger(Callback):
    def __init__(self):
        self.seen = 0

    def on_epoch_end(self, batch, logs={}):
        self.seen += 1
        if self.seen % 500 == 0:
            print('epoch {}: loss = {}, val = {}'.format(self.seen, logs['loss'], logs['val_loss'])) 
            
class FakeDMatrix:
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        self.num = len(data)

    def num_row(self):
        return self.num

    def slice(self, rindex):
        indices = np.zeros(self.num, dtype=np.bool)
        for index in rindex:
            indices[index] = True
        return FakeDMatrix(data=self.data[indices], labels=self.labels[indices])

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [54]:
train_X = pd.read_json('data/train_scaled_with_leaky_and_added_features_cv.json', orient='records')
train_y = pd.read_json('data/train_interest.json')[0]

In [42]:
cols_to_use = ['BoroCode',
 'Day',
 'Month',
 'NTACode',
 'Wday',
 'Yday',
 'Zero_building_id',
 'Zero_manager_id',
 'address_high_frac',
 'address_low_frac',
 'address_med_frac',
 'address_worths',
 'avenue',
 'bathrooms',
 'bc_price',
 'bedrooms',
 'boro_high_frac',
 'boro_low_frac',
 'boro_med_frac',
 'boro_worths',
 'building_high_frac',
 'building_id',
 'building_low_frac',
 'building_med_frac',
 'building_worths',
 'desc_letters_count',
 'desc_words_count',
 'desc_words_length',
 'display_address',
 'east',
 'features_count',
 'hour',
 'latitude',
 'longitude',
 'manager_high_frac',
 'manager_id',
 'manager_low_frac',
 'manager_med_frac',
 'manager_worths',
 'north',
 'nta_high_frac',
 'nta_low_frac',
 'nta_med_frac',
 'nta_worths',
 'other_address',
 'photos_count',
 'price_by_BoroCode',
 'price_by_NTACode',
 'price_by_area',
 'price_by_building_id',
 'price_by_display_address',
 'price_by_manager_id',
 'price_per_bathroom',
 'price_per_bed_and_bath',
 'price_per_bedroom',
 'real_minus_expected_price',
 'real_over_expected_price',
 'south',
 'sparse_24',
 'sparse_2fullbaths',
 'sparse_actualapt',
 'sparse_airconditioning',
 'sparse_allutilitiesincluded',
 'sparse_assignedparkingspace',
 'sparse_attendedlobby',
 'sparse_backyard',
 'sparse_balconies',
 'sparse_balcony',
 'sparse_basementstorage',
 'sparse_bikeroom',
 'sparse_bikestorage',
 'sparse_billiardsroom',
 'sparse_brownstone',
 'sparse_buildingcommonoutdoorspace',
 'sparse_businesscenter',
 'sparse_cable',
 'sparse_catsallowed',
 'sparse_centrala',
 'sparse_centralac',
 'sparse_centralair',
 'sparse_chef',
 'sparse_chefskitchen',
 'sparse_children',
 'sparse_childrensplayroom',
 'sparse_cityview',
 'sparse_commonbackyard',
 'sparse_commongarden',
 'sparse_commonoutdoorspace',
 'sparse_commonparking',
 'sparse_commonroofdeck',
 'sparse_commonterrace',
 'sparse_communityrecreationfacilities',
 'sparse_concierge',
 'sparse_conciergeservice',
 'sparse_condofinishes',
 'sparse_cook',
 'sparse_courtyard',
 'sparse_deck',
 'sparse_decorativefireplace',
 'sparse_diningroom',
 'sparse_dishwasher',
 'sparse_dogsallowed',
 'sparse_doorman',
 'sparse_drycleaningservice',
 'sparse_dryer',
 'sparse_dryerinbuilding',
 'sparse_dryerinunit',
 'sparse_duplex',
 'sparse_eatinkitchen',
 'sparse_elev',
 'sparse_elevator',
 'sparse_elevbldg',
 'sparse_exclusive',
 'sparse_exposedbrick',
 'sparse_fireplace',
 'sparse_fireplaces',
 'sparse_fitness',
 'sparse_fitnesscenter',
 'sparse_flex2',
 'sparse_flex3',
 'sparse_ftdoorman',
 'sparse_fullservicegarage',
 'sparse_fulltimedoorman',
 'sparse_furnished',
 'sparse_garage',
 'sparse_garden',
 'sparse_gourmetkitchen',
 'sparse_granitecountertops',
 'sparse_granitekitchen',
 'sparse_greenbuilding',
 'sparse_guarantorsaccepted',
 'sparse_gutrenovated',
 'sparse_gym',
 'sparse_gyminbuilding',
 'sparse_hardwood',
 'sparse_hardwoodfloors',
 'sparse_healthclub',
 'sparse_highceiling',
 'sparse_highceilings',
 'sparse_highrise',
 'sparse_highspeedinternet',
 'sparse_hirise',
 'sparse_housekeeping',
 'sparse_indoorpool',
 'sparse_inunitwasher',
 'sparse_largelivingroom',
 'sparse_laundry',
 'sparse_laundryinbuilding',
 'sparse_laundryinunit',
 'sparse_laundryonfloor',
 'sparse_laundryroom',
 'sparse_light',
 'sparse_live',
 'sparse_liveinsuper',
 'sparse_liveinsuperintendent',
 'sparse_lndrybldg',
 'sparse_loft',
 'sparse_lounge',
 'sparse_loungeroom',
 'sparse_lowrise',
 'sparse_luxurybuilding',
 'sparse_mailroom',
 'sparse_marblebath',
 'sparse_marblebathroom',
 'sparse_microwave',
 'sparse_midrise',
 'sparse_mrcleanapproved',
 'sparse_multilevel',
 'sparse_new',
 'sparse_newconstruction',
 'sparse_newlyrenovated',
 'sparse_nofee',
 'sparse_nopets',
 'sparse_nursery',
 'sparse_onsitegarage',
 'sparse_onsitelaundry',
 'sparse_onsiteparking',
 'sparse_onsiteparkingavailable',
 'sparse_onsiteparkinglot',
 'sparse_onsitesuper',
 'sparse_ornateprewardetails',
 'sparse_outdoorareas',
 'sparse_outdoorentertainmentspace',
 'sparse_outdoorpool',
 'sparse_outdoorspace',
 'sparse_packageroom',
 'sparse_parking',
 'sparse_parkingspace',
 'sparse_patio',
 'sparse_penthouse',
 'sparse_petfriendly',
 'sparse_pets',
 'sparse_petsallowed',
 'sparse_petsok',
 'sparse_petsonapproval',
 'sparse_photos',
 'sparse_playroom',
 'sparse_pool',
 'sparse_postwar',
 'sparse_prewar',
 'sparse_privatebackyard',
 'sparse_privatebalcony',
 'sparse_privatedeck',
 'sparse_privatelaundryroomoneveryfloor',
 'sparse_privateoutdoorspace',
 'sparse_privateparking',
 'sparse_privateroofdeck',
 'sparse_privateterrace',
 'sparse_publicoutdoor',
 'sparse_reducedfee',
 'sparse_renovated',
 'sparse_residentsgarden',
 'sparse_residentslounge',
 'sparse_roofaccess',
 'sparse_roofdeck',
 'sparse_rooftopdeck',
 'sparse_rooftopterrace',
 'sparse_roomyclosets',
 'sparse_satellitetv',
 'sparse_sauna',
 'sparse_screeningroom',
 'sparse_sharedbackyard',
 'sparse_sharesok',
 'sparse_shorttermallowed',
 'sparse_simplex',
 'sparse_skitchen',
 'sparse_skylight',
 'sparse_splayroom',
 'sparse_sskitchen',
 'sparse_stainlesssteelappliances',
 'sparse_stepstothepark',
 'sparse_storage',
 'sparse_storagefacilitiesavailable',
 'sparse_storageroom',
 'sparse_sublet',
 'sparse_subway',
 'sparse_sundeck',
 'sparse_swimmingpool',
 'sparse_tenantlounge',
 'sparse_terrace',
 'sparse_terraces',
 'sparse_tonsofnaturallight',
 'sparse_valet',
 'sparse_valetparking',
 'sparse_valetservices',
 'sparse_valetservicesincludingdrycleaning',
 'sparse_videointercom',
 'sparse_view',
 'sparse_virtualdoorman',
 'sparse_walkincloset',
 'sparse_wallsofwindows',
 'sparse_washer',
 'sparse_washerinunit',
 'sparse_wheelchairaccess',
 'sparse_wheelchairramp',
 'sparse_wifi',
 'sparse_wifiaccess',
 'sparse_work',
 'street',
 'west',
 'worth_by_BoroCode',
 'worth_by_NTACode',
 'worth_by_building_id',
 'worth_by_display_address',
 'worth_by_manager_id'] + ['layer1_pca_0', 'layer1_pca_1', 'layer1_pca_2'] + ['layer1_ridge_predictions']

In [55]:
train_X = train_X[cols_to_use]

In [44]:
NFOLDS = 5
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.6,
    'subsample':.6,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

In [45]:
b = pd.read_json('data/backup/train.json')
a = pd.read_csv('data/listing_image_time.csv')

In [56]:
train_X['listing_id'] = b.listing_id

In [57]:
a.columns = ['listing_id', 'time_stamp']

In [58]:
length = len(train_X)

In [61]:
train_X = train_X.merge(a, on='listing_id', how='outer')[:length]

In [62]:
train_X

,BoroCode,Day,Month,NTACode,Wday,Yday,Zero_building_id,Zero_manager_id,address_high_frac,address_low_frac,...,worth_by_NTACode,worth_by_building_id,worth_by_display_address,worth_by_manager_id,layer1_pca_0,layer1_pca_1,layer1_pca_2,layer1_ridge_predictions,listing_id,time_stamp
0,2.0,24.0,6.0,45.0,4.0,176.0,0.0,0.0,0.055556,0.888889,...,0.435374,0.000000,0.166667,0.246377,-0.278612,-3.040328,2.660744,1.564045,7211212.0,1478091590
1,0.0,12.0,6.0,90.0,6.0,164.0,0.0,0.0,0.021739,0.811594,...,0.285913,0.125000,0.210145,0.014493,-0.031424,1.914108,2.268480,2.136262,7150865.0,1478129766
2,0.0,14.0,4.0,105.0,3.0,105.0,1.0,0.0,0.033149,0.861878,...,0.339223,0.103884,0.171271,0.000000,-0.274130,0.807725,4.423791,1.908489,6869199.0,1478714534
3,2.0,13.0,4.0,11.0,2.0,104.0,0.0,0.0,0.078973,0.694823,...,0.953271,0.000000,0.000000,0.636364,-0.360539,-8.090306,1.972914,1.115436,6866364.0,1478715526
4,0.0,12.0,4.0,93.0,1.0,103.0,0.0,0.0,0.037037,0.666667,...,0.370454,0.000000,0.370370,0.682927,0.007857,2.283832,-3.447072,1.610766,6859853.0,1478724292
5,0.0,19.0,4.0,92.0,1.0,110.0,0.0,0.0,0.000000,0.847826,...,0.254760,0.166667,0.152174,0.301587,0.013564,3.745476,-2.263358,1.729849,6896146.0,1479081308
6,0.0,6.0,4.0,91.0,2.0,97.0,0.0,0.0,0.000000,1.000000,...,0.349698,0.351648,0.000000,0.000000,-0.137006,1.560335,3.182760,2.039505,6830056.0,1479081320
7,0.0,26.0,4.0,98.0,1.0,117.0,0.0,0.0,0.155172,0.551724,...,0.449625,0.428571,0.603448,0.230769,-0.244400,-1.139707,1.357181,1.633016,6926200.0,1479081334
8,3.0,15.0,4.0,125.0,4.0,106.0,0.0,0.0,0.191794,0.481505,...,1.428571,0.758145,0.709587,0.383744,-0.420045,-13.160018,1.797733,1.147193,6877585.0,1479081362
9,2.0,4.0,6.0,34.0,5.0,156.0,0.0,0.0,0.187965,0.477978,...,0.267606,0.000000,0.710167,0.394958,-0.097738,-0.733334,-2.303526,1.724011,7106422.0,1477650620


In [63]:
ntrain = int(len(train_X) * 0.75)

scaler = StandardScaler()

xtr = train_X[:ntrain]
xtr = pd.DataFrame(scaler.fit_transform(xtr), columns=xtr.columns)
ytr = train_y[:ntrain].values
ytr_one_hot = np.eye(3)[ytr]
xte = train_X[ntrain:]
xte = pd.DataFrame(scaler.transform(xte), columns=xte.columns)
yte = train_y[ntrain:].values
yte_one_hot = np.eye(3)[yte]

In [ ]:
cols = list(train_X.columns)
dtrain = xgb.DMatrix(data=xtr[cols], label=ytr)
dtest = xgb.DMatrix(data=xte[cols])

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])

bst = xgb.train(params, dtrain, best_rounds)
print('train score: ', log_loss(ytr_one_hot, bst.predict(dtrain)))
print('test score: ', log_loss(yte_one_hot, bst.predict(dtest)))

[0]	train-mlogloss:1.09111+1.90263e-05	test-mlogloss:1.09132+4.06664e-05


In [ ]:
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers.core import ActivityRegularization
def create_model(xtr, layer1=256, layer2=32, activation='tanh', activation2=PReLU, dropout=0.75):
    model = Sequential()
    
    model.add(Dense(layer1,
                    input_dim=xtr.shape[1],
                    init = 'he_normal',
                    activation=activation))
                    #kernel_regularizer=regularizers.l2(0.01),
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    #model.add(Dense(64, input_dim=64, W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
    
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(layer2,
                    init = 'he_normal',
                    activation=activation))
                    #activity_regularizer=regularizers.activity_l2(0.005)))
    model.add(BatchNormalization())    
    model.add(Dropout(dropout))
    if activation2 != None:
        model.add(activation2())

    model.add(Dense(3, init = 'he_normal', activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
    return model

In [ ]:
filepath = 'data/nn_weights'

nn_cols = list(train_X.columns)

logger = NBatchLogger()
early_stop = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)
model = create_model(xtr[nn_cols])
model.fit(xtr[nn_cols].values, ytr_one_hot, nb_epoch = 1200, batch_size=1000, verbose = 0, validation_data=[xte[nn_cols].values, yte_one_hot], callbacks=[logger, early_stop, checkpoint])

model = create_model(xtr[nn_cols])
model.load_weights(filepath)
nn_preds = model.predict(xte[nn_cols].values)
print(log_loss(yte_one_hot, nn_preds))

In [ ]:
%%time
et_cols = nn_cols

test = ExtraTreesClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=50)

test.fit(xtr[et_cols], ytr)
            
et_train_predictions = test.predict_proba(xtr[et_cols])
et_predictions = test.predict_proba(xte[et_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, et_predictions), log_loss(ytr_one_hot, et_train_predictions)))
# 0.5989926646715679

In [ ]:
%%time
rf_cols = nn_cols

test = RandomForestClassifier(
    n_estimators=100,
    criterion='entropy',
    max_features='auto',
    min_samples_split=50)

test.fit(xtr[rf_cols], ytr)
            
rf_train_predictions = test.predict_proba(xtr[rf_cols])
rf_predictions = test.predict_proba(xte[rf_cols])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, rf_predictions), log_loss(ytr_one_hot, rf_train_predictions)))
# 0.5814357939419752

In [ ]:
%%time
test = KNeighborsClassifier(n_neighbors=500)
test.fit(xtr[knn_features], ytr)

knn_train_predictions = test.predict_proba(xtr[knn_features])
knn_predictions = test.predict_proba(xte[knn_features])
print ('{} (test loss {})'.format(log_loss(yte_one_hot, knn_predictions), log_loss(ytr_one_hot, knn_train_predictions)))